In [30]:
import tiktoken # for token counting
import numpy as np
from collections import defaultdict
import json

In [47]:
data_path = "50_conversation_json_file.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 50
First example:
{'role': 'system', 'content': 'This assistant is knowledgeable about Wukuf, Umrah preparation, and other rituals details'}
{'role': 'assistant', 'content': "Assalamu'alaikum! Bagaimana kabar Bapak/Ibu? Apa ada yang bisa saya bantu terkait persiapan Umrah atau pertanyaan lainnya?"}
{'role': 'user', 'content': "Wa'alaikumsalam. Alhamdulillah, saya baik. Saya ingin tahu, bagaimana sih cara melaksanakan wukuf di Arafah?"}
{'role': 'assistant', 'content': 'Alhamdulillah, senang mendengarnya. Wukuf di Arafah adalah puncak ibadah haji yang dilakukan dengan berdiam diri di Padang Arafah pada 9 Dzulhijjah, mulai dari tergelincir matahari sampai terbenam. Wukuf ini sangat penting karena tanpa melaksanakannya, haji tidak sah. Ada hal lain tentang wukuf di Arafah yang ingin Bapak/Ibu ketahui?'}
{'role': 'user', 'content': 'Terima kasih informasinya. Kalau misalnya saya sakit selama umrah, apa yang harus saya lakukan?'}
{'role': 'assistant', 'content': 'Jika Bapak/Ib

In [48]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [49]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [50]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 24, 24
mean / median: 24.0, 24.0
p5 / p95: 24.0, 24.0

#### Distribution of num_total_tokens_per_example:
min / max: 1484, 1484
mean / median: 1484.0, 1484.0
p5 / p95: 1484.0, 1484.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 1122, 1122
mean / median: 1122.0, 1122.0
p5 / p95: 1122.0, 1122.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


### Cost Estimation

In [51]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~74200 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~222600 tokens


In [52]:
# Prices per million tokens for each model
price_per_million_gpt35 = 8.00
price_per_million_davinci = 6.00
price_per_million_babbage = 0.40

# Total tokens charged during training
tokens_charged = n_epochs * n_billing_tokens_in_dataset

# Calculate cost per token for each model
cost_per_token_gpt35 = price_per_million_gpt35 / 1000000
cost_per_token_davinci = price_per_million_davinci / 1000000
cost_per_token_babbage = price_per_million_babbage / 1000000

# Calculate total training cost for each model
total_cost_gpt35 = cost_per_token_gpt35 * tokens_charged
total_cost_davinci = cost_per_token_davinci * tokens_charged
total_cost_babbage = cost_per_token_babbage * tokens_charged

print('Estimated Total Cost For Training Using GPT3.5: $', total_cost_gpt35)
print('Estimated Total Cost For Training Using Davinci: $', total_cost_davinci)
print('Estimated Total Cost For Training Using Cost Babbage: $', total_cost_babbage)

Estimated Total Cost For Training Using GPT3.5: $ 1.7808
Estimated Total Cost For Training Using Davinci: $ 1.3356000000000001
Estimated Total Cost For Training Using Cost Babbage: $ 0.08904000000000001


## Fine Tuning Model

In [55]:
from openai import OpenAI

In [56]:
client = OpenAI(api_key='')

training_file = client.files.create(
  file=open("50_conversation_json_file.jsonl", "rb"),
  purpose="fine-tune"
)

In [57]:
print(training_file.id)

file-OoeIlkAly7hfozZJZDrh9PN9


In [76]:
# Create Fine-Tuning Job
suffix_name = "umroh_ai_mvp_29jul"
response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-3.5-turbo-0125",
    suffix=suffix_name,
)

In [77]:
training_file.id

'file-OoeIlkAly7hfozZJZDrh9PN9'

In [78]:
response.status

'validating_files'